In [1]:
import torch
import torch.utils.data
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
import numpy as np
import h5py
from data_utils import get_data

In [2]:
# Load data from all .mat files, combine them, eliminate EOG signals, shuffle and 
# seperate training data, validation data and testing data.
# Also do mean subtraction on x.

data = get_data('project_datasets')
for k in data.keys():
    print('{}: {} '.format(k, data[k].shape))

X_train: (2458, 22, 1000) 
y_train: (2458,) 
X_val: (50, 22, 1000) 
y_val: (50,) 
X_test: (50, 22, 1000) 
y_test: (50,) 


In [3]:
print(data.get('X_test'))

[[[ -6.63188441 -16.93847259 -20.64440411 ...,  -6.92989886  -5.61507546
    -0.30439111]
  [ -5.19079918 -19.50032022 -18.81617025 ...,  -4.73688434  -2.12519547
    -2.7116296 ]
  [ -8.10713741 -18.5641035  -22.42398863 ...,  -9.0166556   -6.58740671
    -2.68685796]
  ..., 
  [ -7.07040388 -14.45203243 -14.61367378 ...,  -6.60468925  -8.34251758
    -8.87728288]
  [-10.32281787 -15.00785461 -14.88276482 ...,  -6.48935316  -8.29680825
    -8.72142224]
  [ -4.47264036 -11.90514312 -14.4495493  ...,  -0.20653621  -3.16894134
    -5.63772153]]

 [[ 17.92866247  18.99902741  12.99817401 ...,  -0.43575824   0.34195579
    -2.01337549]
  [ 15.8541227   13.5563204    9.84593912 ...,   9.13030316   7.83574203
     5.97977665]
  [ 16.20926884  17.32456837  11.60921449 ...,   5.0458444    5.76610891
     4.14907954]
  ..., 
  [ 16.708893    17.72570195  14.78085747 ...,   7.60429513   5.32935742
     1.32779524]
  [ 11.79632275  11.99409851   9.62895393 ...,   3.61806871   0.68756675
    -2.91

In [4]:
# class flatten to connect to FC layer
class Flatten(nn.Module):
    def forward(self, x):
        N, C, H = x.size() # read in N, C, H
        return x.view(N, -1)

In [5]:
# turn x and y into torch type tensor
dtype = torch.FloatTensor

X_train = Variable(torch.Tensor(data.get('X_train'))).type(dtype)
y_train = Variable(torch.Tensor(data.get('y_train'))).type(dtype)
X_val = Variable(torch.Tensor(data.get('X_val'))).type(dtype)
y_val = Variable(torch.Tensor(data.get('y_val'))).type(dtype)
X_test = Variable(torch.Tensor(data.get('X_test'))).type(dtype)
y_test = Variable(torch.Tensor(data.get('y_test'))).type(dtype)

In [6]:
# set up sequential model

model = nn.Sequential(
                      nn.Conv1d(22, 32, kernel_size=21, stride=1),
                       nn.ReLU(inplace=True),
                       nn.BatchNorm1d(num_features=32),
                       nn.MaxPool1d(kernel_size=4, stride=4),
                      Flatten(),
                      nn.Linear(7840, 400),
                      nn.ReLU(inplace=True),
                      nn.Linear(400, 4)
)

model.type(dtype)
loss_fn = nn.CrossEntropyLoss().type(dtype)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
# train through several iterations

for t in range(1):
    
    # calculate loss
    y_pred = model(X_train)
    loss = loss_fn(y_pred, y_train.type(torch.LongTensor))
    
    # backpropagation
    model.zero_grad()
    loss.backward()
    
    # update parameters
    optimizer.step()
    
    print(t,loss)

In [ ]:
np.set_printoptions(threshold=np.nan)
y_temp = y_pred.data.numpy()
#print(y_temp)
np.isnan(np.sum(y_temp))
